In [8]:
from pandas import DataFrame, read_csv
from os.path import join, basename
from os.path import splitext, join
filename = r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_GFF3_genes_only.gff3"

In [8]:
df = read_csv(r"D:\josiah\Documents\Research\Arabidopsis\functional_annotation.csv",sep='\t', error_bad_lines=False)# fields='Gene_Family	Gene_Name	Genomic_Locus(BAC)	Genomic_Locus_Tag	Protein_Function'.split())

In [9]:
df.head()

,Gene_Family,Gene_Name,Genomic_Locus(BAC),Genomic_Locus_Tag,Protein_Function
0,IQD Protein Family,IQD1,NaN,At3g09710,NaN
1,IQD Protein Family,IQD2,NaN,At5g03040,NaN
2,IQD Protein Family,IQD3,NaN,At3g52290,NaN
3,IQD Protein Family,IQD4,NaN,At2g26410,NaN
4,IQD Protein Family,IQD5,NaN,At3g22190,NaN


In [107]:
len(df)

8693

In [106]:
sum(df['Gene_Family'].notnull())

8693

In [22]:
sum(df['Protein_Function'].str.contains('putative')==False)

3723

In [26]:
sum(df['Protein_Function'].notnull() & (df['Protein_Function'].str.contains('hypothetical protein')==False))

5804

In [25]:
6112 /28775

0.21240660295395308

In [36]:
8472 /28775

0.2944222415291051

In [46]:
len(set(tags))

7647

In [47]:
len(set([str(n) for n in tags if n]))

7647

In [51]:
6805/28775

0.2364900086880973

In [48]:
len(set([str(n).upper() for n in tags if n]))

6805

# Filter out to only genes with mentions in functional_annotation.csv

In [45]:
tag = df['Genomic_Locus_Tag'].notna()
tags = df[tag]['Genomic_Locus_Tag']
sum(tag), len(tag)

(8472, 8693)

In [55]:
upper_case_loci = set([str(n).upper() for n in tags])
len(upper_case_loci)

6805

In [86]:

filtered_path = splitext(filename)[0] + '_with_functions' + splitext(filename)[1]
with open(filename) as file:
    with open(filtered_path, 'w') as outfile:
        for line in file.readlines():
            attributes = line.split('\t')[8].split(';')
            locus = attributes[0][3:]
            if locus.upper() in upper_case_loci or name.upper() in upper_case_loci:
                outfile.write(line)

6652 lines

#### Create Gene function Lookup

In [116]:
gene_function = {}
for index, row in df[tag].iterrows():
    try: 
        gene = row['Genomic_Locus_Tag'].upper()
    except AttributeError:
        print(row['Genomic_Locus_Tag'])
    function = row['Protein_Function'] if str(row['Protein_Function']) != 'nan' else row['Gene_Family']
    forbidden = ['\uffa0', 'basic', 'hypothetical protein', 'unknown protein', 'putative', 'hypothetical  protein']
    for bad in forbidden:
        function = function.replace(bad, '')
    if not function.strip() or str(function) == 'nan':
        function = row['Gene_Family'].replace('basic', '')
    gene_function[gene] = function
#     break
gene_function

{'AT3G47090': 'receptor kinase-like protein',
 'AT1G27400': 'L17 in small subunit',
 'AT3G19553': 'amino acid permease, ',
 'AT1G78600': 'C2C2-CO-like Transcription Factor Family',
 'AT5G55360': 'wax synthase-like protein',
 'AT5G08130': 'bHLH Transcription Factor Family',
 'AT3G11940': 'S5 in small subunit',
 'AT1G28370': 'AP2-EREBP Transcription Factor Family',
 'AT1G62660': 'beta-fructosidase',
 'AT4G19210': 'RNase L inhibitor-like protein',
 'AT4G37260': 'MYB Transcription Factor Family',
 'AT3G13580': 'L7 in small subunit',
 'AT4G34030': '3-methylcrotonyl-CoA carboxylase non-biotinylated subunit (MCCB)',
 'AT2G46950': ' cytochrome P450',
 'AT3G27970': 'C2H2 Transcription Factor Family',
 'AT2G45110': 'beta-expansin pollen allergen protein',
 'AT4G24750': ' protein',
 'AT4G37640': 'plasma membrane-type calcium ATPase (ACA2)',
 'AT1G56680': 'chitinase, ',
 'AT3G09630': 'L4 in small subunit',
 'AT5G15200': 'S9 in small subunit',
 'AT1G48490': 'protein kinase, similar to IRE',
 'AT5G4

** ----------------- TODO: Come back and make the name pretty ---------------------**

### Decorate Annotation with functions

In [117]:
with open(filtered_path, 'r') as genes:
    with open(r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_annotated_genes.gff", 'w') as final_annotation:
        for line in genes.readlines():
            parts = line.split('\t')
            locus = parts[8].split(';')[0][3:]
            function = gene_function[locus]
            if locus in gene_function:
                final_annotation.write('\t'.join(parts[:8] + ['Name=' + function]) +'\n')

In [113]:
df[df['Genomic_Locus_Tag'] =='At1g01570']

,Gene_Family,Gene_Name,Genomic_Locus(BAC),Genomic_Locus_Tag,Protein_Function
4611,Glycosyltransferase Gene Families,NaN,NaN,At1g01570,hypothetical protein


In [118]:
'AT1G01020' in upper_case_loci

False

# Repeat the Process for ATH_GO_SLIM

In [3]:
go = read_csv(r"D:\josiah\Documents\Research\Arabidopsis\ATH_GO_GOSLIM.txt",sep='\t', error_bad_lines=False,
             usecols=['locus name', 'GO term','GOslim term', 'Date annotated'])
go.head()

,locus name,GO term,GOslim term,Date annotated
0,AT1G01010,"regulation of transcription, DNA-templated",other cellular processes,2018-04-02
1,AT1G01010,"regulation of transcription, DNA-templated","transcription,DNA-dependent",2018-04-02
2,AT1G01010,DNA-binding transcription factor activity,transcription factor activity,2003-06-06
3,AT1G01010,"regulation of transcription, DNA-templated",other cellular processes,2018-09-12
4,AT1G01010,integral component of membrane,other membranes,2018-09-04


In [4]:
sum(go['GO term'].notnull() & go['locus name'].notnull()) ,len(go)

(349103, 349103)

Terrific!  No missing data

In [5]:
go_upper = {cell.upper() for cell in go['locus name']}
len(go_upper)

30828

30,828 is slightly more genes than TAIR10 annotation...  because there are non-coding regions...

In [6]:
go_function = {}
for index, row in go.iterrows():
    gene = row['locus name']
    function = row['GO term']
    forbidden = ['biological_process', 'cellular_component', 'molecular_function',  'extracellular region', 'nucleus',]
    #            'mitochondrion',  'cytoplasm', 'cytosol', 'chloroplast', 'membrane', 'plasma membrane']

    if function not in forbidden:
        if gene not in go_function or len(go_function[gene]) < len(function):  # use the longest
            go_function[gene] = function
len(go_function)

25302

In [139]:
list(go_function.values())[:1000]

['endoplasmic reticulum',
 'leaf morphogenesis',
 'ARF guanyl-nucleotide exchange factor activity',
 'reactive oxygen species metabolic process',
 'hydrolase activity, acting on ester bonds',
 'aspartic-type endopeptidase activity',
 'proteasome-mediated ubiquitin-dependent protein catabolic process',
 'ribosomal subunit export from nucleus',
 'regulation of cell population proliferation',
 'killing of cells of other organism',
 'vegetative to reproductive phase transition of meristem',
 'photosystem II oxygen evolving complex',
 'positive regulation of transcription, DNA-templated',
 'negative regulation of transcription, DNA-templated',
 'chitinase activity',
 'chaperonin-containing T-complex',
 'methyltransferase activity',
 'ATP-dependent helicase activity',
 'endopeptidase inhibitor activity',
 'RNA modification',
 'strictosidine synthase activity',
 'protein serine/threonine kinase activity',
 'positive regulation of development, heterochronic',
 'metal ion transmembrane transpor

In [9]:
with open(filename, 'r') as genes:
    with open(r"D:\josiah\Documents\Research\Arabidopsis\GO_annotation.gff", 'w') as final_annotation:
        for line in genes.readlines():
            parts = line.split('\t')
            locus = parts[8].split(';')[0][3:]
            if locus in go_function:
                function = go_function[locus]
                final_annotation.write('\t'.join(parts[:8] + ['Name=%s;ID=%s'% (function, locus) ]) +'\n')

22,103 genes

In [10]:
from DNASkittleUtils.Contigs import Contig, read_contigs, write_contigs_to_file

In [11]:
chromosomes = read_contigs(r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_chr_all.fas")
for chrm in chromosomes:
    chrm.name = chrm.name.split()[0]
    write_contigs_to_file(join(r"D:\josiah\Documents\Research\Arabidopsis\Chromosomes", chrm.name + '.fa'), [chrm])

Done writing  1 contigs and 30,427,671bp
Done writing  1 contigs and 19,698,289bp
Done writing  1 contigs and 23,459,830bp
Done writing  1 contigs and 18,585,056bp
Done writing  1 contigs and 26,975,502bp
Done writing  1 contigs and 366,924bp
Done writing  1 contigs and 154,478bp


In [25]:
from functools import reduce
reduce(int.__mul__,[3,3,3,3,3,15]) + 200

3845

In [26]:
reduce(int.__mul__, [5,3,3,3,3,]) # coil_height

405